In [14]:
import os
from dotenv import load_dotenv
from pyairtable import Table

load_dotenv()

APITOKEN = os.getenv("APITOKEN")
APPID = os.getenv("APPID")
UABList = os.getenv("UABList")

testTable = Table(APITOKEN, APPID, UABList)

for each in testTable.all():
    print(each["fields"]["Name"])



uab-ab-victory
uab-cat
uab-janitor-animated
uab-janitor-static
uab-ab-static
uab-dog
